# Coding A Rank Order Assignment Algorithm

The goal of the project was to write a piece of software to matches N Doctors with K Hospitals with open residency positions. Each Doctor provides a ranked list of their preference for Hospitals, without Hospitals ranking the Doctors.

### Section 1: Background
Blabla Hungarian Algorithm.  PUT ACTUAL STUFF HERE once we finish README.md

For a more in depth look at the Hungarian Algorithm, you can reference <a href="https://www.topcoder.com/thrive/articles/Assignment%20Problem%20and%20Hungarian%20Algorithm" target="_parent"> this article on topcoder.com.

### Section 2: The Data

We assume our data has been collected into a spreadsheet (a CSV file) and organized so that each doctor has their own row, which is itself populated with preference rankings [1, K] for each hospital in it's corresponding column.  

We load this in as a **PANDAS** dataframe.

In [2]:
import HungarianAlgoImportData as HAID

df = HAID.import_data('BDD Test.csv')
print(df)

  Hospital  H1  H2  H3
0      Num   2   2   1
1       D1   1   2   3
2       D2   2   1   3
3       D3   3   2   1
4       D4   1   3   2


From this dataframe, we extract the details provided for each hospital and strip the dataframe of its row and column labels to allow us to work with just the cost matrix. 

In [7]:
hi = HAID.get_hospital_info(df)

prepped_df = HAID.prep_data(df)
prepped_array = prepped_df.to_numpy(copy = True)

print(f'get_hospital_info returns the number of positions available per hospital\n {hi}')
print(f'At the end of the importing process we are left with the 2D array:\n {prepped_array}')

get_hospital_info returns the number of positions available per hospital
   Hospital  H1  H2  H3
0      Num   2   2   1
At the end of the importing process we are left with the 2D array:
 [[1 2 3 1]
 [2 1 3 2]
 [3 2 1 3]
 [1 3 2 1]]


### Section 3: Reduce the Matrix

With our cost matrix created we then begin the steps of the Hungarian Algorithm.  

As previously explained, the first step is to reduce the matrix by subtracting te minimum value of each row from each row and repeating that for each column. 

In [10]:
import Hungarian_Alg_Steps as HAS

row_reduced_matrix = HAS.step1_row_reduction(prepped_array)
print(f'Our original cost matrix was:\n {prepped_array}\n\n and following our row reduction we are given:\n {row_reduced_matrix}')

reduced_matrix = HAS.step2_col_reduction(row_reduced_matrix)
print(f'\nAfter column reduction, we obtain our final reduced matrix:\n {reduced_matrix}')

Our original cost matrix was:
 [[1 2 3 1]
 [2 1 3 2]
 [3 2 1 3]
 [1 3 2 1]]

 and following our row reduction we are given:
 [[0. 1. 2. 0.]
 [1. 0. 2. 1.]
 [2. 1. 0. 2.]
 [0. 2. 1. 0.]]

After column reduction, we obtain our final reduced matrix:
 [[0. 1. 2. 0.]
 [1. 0. 2. 1.]
 [2. 1. 0. 2.]
 [0. 2. 1. 0.]]


### Section 4: Crossing Out the Zeros

I don't feel like explaining this right now.  I'll come back to it tomorrow or smth